<a href="https://colab.research.google.com/github/testli-ai/testli-ai-fine-tuning/blob/main/testliai_generate_exam_mistral_7b_instruct_v0_3_bnb_4bit_GGUF_q4_k_m_v0_0_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# next model name: testliai-main/testliai-generate-exam-mistral-7b-instruct-v0.3-bnb-4bit-GGUF-q4_k_m-v0.0.2

In [ ]:
!pip install transformers==4.47.1 datasets==3.2.0 torch==2.5.1+cu124 torchvision==0.20.1+cu124
!pip install tokenizers==0.21.0 bitsandbytes==0.45.1 peft==0.14.0 accelerate==1.2.1
!pip install trl==0.12.0 unsloth ipykernel==5.5.6
!pip install outlines

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.3/343.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.6/220.6 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 913.7/913.7 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 103.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 32.0 MB/s eta 0:00:00


In [ ]:
from tokenize import tokenize
from datasets import load_dataset, Dataset, DatasetDict
from google.colab import userdata
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,TrainingArguments, logging, pipeline
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
import torch
from trl import SFTTrainer
from google.colab import userdata
from datasets import load_dataset, Dataset, DatasetDict
from unsloth import FastLanguageModel



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
# dataset = load_dataset('json', data_files='converted_exam.json')
# dataset['train']

In [ ]:
hf_token = userdata.get("thongtestli")

# Khởi tạo tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    token=hf_token,
)


In [ ]:
# import numpy as np
# from datasets import Dataset, DatasetDict

# # Hàm lấy Top K đề thi dựa trên số lượng token
# def get_top_k_exams(dataset, token_counts, k):
#     k = min(k, len(token_counts))  # Đảm bảo k không vượt quá số lượng dữ liệu thực tế
#     top_k_indices = [int(i) for i in np.argsort(token_counts)[-k:][::-1]]  # Chuyển numpy.int64 thành int

#     # Trích xuất dữ liệu theo chỉ số đã sắp xếp
#     top_k_data = {
#         "exam_id": [],
#         "sections": []
#     }

#     for i in top_k_indices:
#         if "output" in dataset[i] and isinstance(dataset[i]["output"], list):  # Kiểm tra output có tồn tại và là danh sách
#             exam_sections = []
#             for output_item in dataset[i]["output"]:
#                 if isinstance(output_item, dict) and "sections" in output_item:  # Kiểm tra output_item là dict có "sections"
#                     for section in output_item["sections"]:
#                         if all(key in section for key in ["section_name", "section", "passage_text", "questions"]):
#                             exam_sections.append({
#                                 "section_name": section["section_name"],
#                                 "section": section["section"],
#                                 "passage_text": section["passage_text"],
#                                 "questions": section["questions"]
#                             })

#             if exam_sections:  # Chỉ thêm vào danh sách nếu có dữ liệu
#                 top_k_data["exam_id"].append(i)  # Lưu ID của đề thi
#                 top_k_data["sections"].append(exam_sections)  # Lưu danh sách sections

#     return Dataset.from_dict(top_k_data)

# # Giá trị k (số lượng đề thi muốn giữ lại)
# k = 200

# # Tạo DatasetDict mới để lưu trữ kết quả
# filtered_datasets = {}

# # Áp dụng cho tất cả các tập dữ liệu trong DatasetDict
# for split in dataset.keys():  # Duyệt qua các tập dữ liệu như "train", "validation", "test"
#     # Tính tổng số lượng token trong tất cả các đoạn passage_text của từng đề thi
#     combined_token_counts = []

#     for item in dataset[split]:
#         total_tokens = 0
#         if "output" in item and isinstance(item["output"], list):  # Đảm bảo output là danh sách
#             for output_item in item["output"]:
#                 if isinstance(output_item, dict) and "sections" in output_item:  # Kiểm tra output_item là dict có "sections"
#                     total_tokens += sum(len(section["passage_text"].split()) for section in output_item["sections"] if "passage_text" in section)

#         combined_token_counts.append(total_tokens)

#     # Lọc Top K đề thi dựa trên số token
#     top_k_dataset = get_top_k_exams(dataset[split], combined_token_counts, k)

#     # Lưu vào DatasetDict mới
#     filtered_datasets[split] = top_k_dataset

# # Chuyển thành DatasetDict hoàn chỉnh
# filtered_dataset = DatasetDict(filtered_datasets)

# # In kết quả
# print(f"Tập dữ liệu sau khi lọc Top {k} đề thi: {filtered_dataset}")


In [ ]:
# hf_token = userdata.get('thongtestli')
# dataset['train'].to_pandas()
# dataset['train'].push_to_hub("English_exam03.03", token=hf_token)

In [ ]:
base_model = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit"
dataset_name = "testliai-main/English_exam03.03"
new_model = "testliai-main/testliai-generate-exam-mistral-7b-instruct-v0.3-bnb-4bit-GGUF-q4_k_m"

In [ ]:
from datasets import load_dataset
dataset = load_dataset(dataset_name)
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 176
    })
})

In [ ]:
def formatting_prompts_func(examples):
    prompts = []

    for exam in examples:  # Duyệt qua từng đề thi trong danh sách output
        if isinstance(exam, dict):  # Kiểm tra xem exam có phải là dict không
            level = exam.get("input", {}).get("level", "unspecified")  # Lấy mức độ khó của đề (Easy, Intermediate, Advanced)
        else:
            continue  # Bỏ qua nếu exam không phải là dict

        # Bắt đầu prompt với hướng dẫn rõ ràng
        prompt = f"### English Exam Generator ###\n\n"
        prompt += f"Generate a brand-new English exam at the {level} level.\n"
        prompt += f"Follow the same format as learned.\n\n"
        prompt += f"### English Exam ###\n\n"

        for section in exam.get("sections", []):  # Duyệt qua từng phần của đề thi
            prompt += f"**Section: {section.get('section_name', 'Unnamed Section')}**\n"

            passage_text = section.get("passage_text", "").strip()
            if passage_text:  # Nếu có đoạn văn thì thêm vào prompt
                prompt += f"Passage:\n{passage_text}\n\n"

            prompt += "Questions:\n"
            for question in section.get("questions", []):
                prompt += f"- {question.get('question_text', 'No question text available')}\n"
                for idx, option in enumerate(question.get("options", [])):
                    prompt += f"  {idx}. {option}\n"
                prompt += "\n"

        prompt += f"{EOS_TOKEN}\n"  # Thêm token kết thúc để mô hình nhận diện
        prompts.append(prompt)

    return {"prompt": prompts}

# Đảm bảo rằng EOS_TOKEN được định nghĩa
EOS_TOKEN = "<|endoftext|>"  # Nếu tokenizer.eos_token không khả dụng, có thể sử dụng chuỗi này

# Tạo dataset mới với định dạng prompt
if "dataset" in globals():
    dataset = dataset.map(formatting_prompts_func)


In [ ]:
!nvidia-smi

Mon Mar  3 13:13:03 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   29C    P0             41W /  400W |       5MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'prompt'],
        num_rows: 176
    })
})

In [ ]:
hf_token = userdata.get('thongtestli')
max_seq_length = 6144
dtype=torch.float16
load_in_4bit = False

# Load the model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=base_model,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    token=hf_token,
)

# Configure tokenizer
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({'eos_token': tokenizer.eos_token, 'bos_token': tokenizer.bos_token})

# Configure the model with LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Suggested values: 8, 16, 32, 64, 128
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_alpha=16,
    lora_dropout=0,  # Optimized when set to 0
    bias="none",     # Optimized with "none"
    use_gradient_checkpointing=True,  # Set "True" or a specific method supported
    random_state=3407,
    use_rslora=False,   # Optional: Rank-stabilized LoRA
    loftq_config=None,  # Optional: LoFT Quantization config
)


==((====))==  Unsloth 2025.2.15: Fast Mistral patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Unsloth 2025.2.15 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
def format_output_text(example):
    """Extract text from the 'output' field into a usable format."""
    sections_text = []

    for output in example.get("output", []):
        for section in output.get("sections", []):
            text = f"Section: {section.get('section_name', 'Unnamed Section')}\n"
            if section.get("passage_text"):
                text += f"Passage:\n{section['passage_text']}\n\n"
            text += "Questions:\n"
            for q in section.get("questions", []):
                text += f"- {q.get('question_text', '')}\n"
                for idx, option in enumerate(q.get("options", [])):
                    text += f"  {idx}. {option}\n"
            sections_text.append(text)

    return {"formatted_output": "\n".join(sections_text)}

# Apply transformation
dataset = dataset.map(format_output_text)

# Now use "formatted_output" for training
dataset_text_field="formatted_output"


In [ ]:
for i in range(5):
    print(dataset['train'][i])


{'input': {'level': 'easy', 'prompt': 'Generate a new English exam'}, 'output': [{'exam_number': 1, 'sections': [{'passage_text': 'Water is very important for life. It has many special ____ (1)____. It freezes and becomes ice which is____ (2)____ than water that stays liquid. Unlike many liquids, water ____(3)_____ when it becomes ice. Ice is essential _ _(4)_ _ for most natural creatures to survive in colder months. Because water is a solvent, it can ____ (5)_____ many different substances. Also, it takes quite a long time for its _____ (6)____ to change.', 'questions': [{'correct_option': 1, 'options': ['things', 'properties', 'materials', 'shapes'], 'question_text': 'What should be put in the blank (number 1)?'}, {'correct_option': 3, 'options': ['cooler', 'smaller', 'hotter', 'larger'], 'question_text': 'What is number (2)?'}, {'correct_option': 1, 'options': ['melts', 'expands', 'freezes', 'shrinks'], 'question_text': 'Choose only one option to fill the blank labeled (3)?'}, {'cor

In [ ]:
# Check for GPU availability and move the model accordingly
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to_empty(device=device)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset['train'],
    dataset_text_field="formatted_output",
    max_seq_length=6144,
    dataset_num_proc=4,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        num_train_epochs=5,
        warmup_steps=50,
        learning_rate=1e-4,
        fp16=True,
        bf16=False,
        logging_steps=50,
        save_steps=10,
        save_total_limit=2,
        optim="adamw_torch",
        weight_decay=0.01,
        output_dir="outputs",
        seed=3407,
        report_to="none",
    ),
)


In [ ]:
#train
trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 176 | Num Epochs = 5
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 4
\        /    Total batch size = 16 | Total steps = 55
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
50,10.397200


TrainOutput(global_step=55, training_loss=10.397210415926846, metrics={'train_runtime': 1262.3054, 'train_samples_per_second': 0.697, 'train_steps_per_second': 0.044, 'total_flos': 2.1405627007382323e+17, 'train_loss': 10.397210415926846, 'epoch': 5.0})

In [ ]:
# model.save_pretrained("mistral_7B_instruct")
# tokenizer.save_pretrained("mistral_7B_instruct")

In [ ]:
model.save_pretrained_gguf( new_model, tokenizer, quantization_method="f16")

Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 14.5G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 60.78 out of 83.48 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [00:00<00:00, 62.42it/s]


Unsloth: Saving tokenizer... Done.
Done.


Unsloth: Converting mistral model. Can use fast conversion = True.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['f16'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at testliai-main/testliai-generate-exam-mistral-7b-instruct-v0.3-bnb-4bit-GGUF-q4_k_m into f16 GGUF format.
The output location will be /content/testliai-main/testliai-generate-exam-mistral-7b-instruct-v0.3-bnb-4bit-GGUF-q4_k_m/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: testliai-generate-exam-mistral-7b-instruct-v0.3-bnb-4bit-GGUF-q4_k_m
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: lo

In [ ]:
model.save_pretrained_gguf( new_model, tokenizer, quantization_method="q4_k_m")

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 61.14 out of 83.48 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [00:00<00:00, 72.80it/s]


Unsloth: Saving tokenizer... Done.
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at testliai-main/testliai-generate-exam-mistral-7b-instruct-v0.3-bnb-4bit-GGUF-q4_k_m into f16 GGUF format.
The output location will be /content/testliai-main/testliai-generate-exam-mistral-7b-instruct-v0.3-bnb-4bit-GGUF-q4_k_m/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: testliai-generate-exam-mistral-7b-instruct-v0.3-bnb-4bit-GGUF-q4_k_m
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight ma

In [ ]:
model.push_to_hub_gguf(new_model, token=hf_token, tokenizer = tokenizer, quantization_method = "q4_k_m")

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 61.25 out of 83.48 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [00:00<00:00, 70.39it/s]


Unsloth: Saving tokenizer... Done.
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at testliai-main/testliai-generate-exam-mistral-7b-instruct-v0.3-bnb-4bit-GGUF-q4_k_m into f16 GGUF format.
The output location will be /content/testliai-main/testliai-generate-exam-mistral-7b-instruct-v0.3-bnb-4bit-GGUF-q4_k_m/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: testliai-generate-exam-mistral-7b-instruct-v0.3-bnb-4bit-GGUF-q4_k_m
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight ma

unsloth.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

Saved GGUF to https://huggingface.co/testliai-main/testliai-generate-exam-mistral-7b-instruct-v0.3-bnb-4bit-GGUF-q4_k_m


In [ ]:
!pip install --upgrade transformers torch outlines


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 77.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Successfully uninstalled transformers-4.47.1


In [ ]:
import json
import torch
from typing import List
from pydantic import BaseModel, Field
from outlines import generate, models
from google.colab import userdata
from transformers import AutoModelForCausalLM, AutoTokenizer, logging, BitsAndBytesConfig

logging.set_verbosity_error()  # Suppress warnings
model_name = "testliai-main/testliai-generate-exam-mistral-7b-instruct-v0.3-bnb-4bit"
# Check for GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"

# Model and authentication setup

hf_token = userdata.get('thongtestli')  # Hugging Face token

# Enable 4-bit quantization to reduce VRAM usage
quantization_config = BitsAndBytesConfig(load_in_4bit=True)

# Load model with 4-bit precision to reduce memory usage
llm = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    token=hf_token
).to(device)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)

# Define exam schema
from typing import List, Optional
from pydantic import BaseModel, Field

class Question(BaseModel):
    question_text: str = Field(..., description="The question text.")
    options: List[str] = Field(..., min_items=4, max_items=4, description="Four multiple-choice answer options.")
    correct_option: int = Field(..., ge=0, le=3, description="Index of the correct answer (0-3).")

class ExamSection(BaseModel):
    section_name: str = Field(..., description="The name of the exam section.")
    section: int = Field(..., description="The section ID.")
    passage_text: Optional[str] = Field(None, description="The passage text, if applicable.")
    questions: List[Question] = Field(..., description="List of questions in the section.")

class Exam(BaseModel):
    exam_number: int
    sections: List[ExamSection] = Field(..., min_items=3, max_items=6, description="List of exam sections.")

# Clear GPU memory before running
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

# Instantiate Outlines Transformers model
model = models.Transformers(llm, tokenizer)

# Limit token generation to prevent OOM errors
generator = generate.json(model, Exam)

# Define prompt
text = """### English Exam Generator ###
Generate a brand-new English exam at the easy level.
Follow the same format as learned.

### English Exam ###
"""

# Run generation safely
with torch.no_grad():  # Prevents unnecessary computation graph storage
    result3 = generator(text)

# Clear GPU memory after running
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

# Print the formatted JSON output
print(result3.model_dump_json(indent=4))


adapter_config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.67M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

AttributeError: 'MistralForCausalLM' object has no attribute 'max_seq_length'